In [12]:
import numpy as np
from qiskit.visualization import array_to_latex=

In [52]:
import os
os.chdir('/Users/ngdnhtien/Research/Codespace/PulsatingPulseShop/')

# Supporting functions

In [2]:
def sum_index(i):
    '''
        Returning sum of index
    '''
    return 2**(i+1)-2

# Hadamard gate and S gate $\langle S, X\rangle = C_3$ 

In [3]:
d = 3 # d-dimensional Lie group, here Clifford C_3 d=3
omega = np.exp(2 * np. pi* 1j / d) # root of unity

In [4]:
Hdm = 1/np.sqrt(3) * np.array([
    [1, 1       ,   1        ],
    [1, omega   ,   omega**2 ],
    [1, omega**2,   omega    ]
])

Sdg = np.array([
    [1,     0,    0        ],
    [0,     1,    0        ],
    [0,     0,    omega    ]
])

# Generation

## Generate the group with abstract elements

Basically, because $H_3$ and $S_3$ generate Clifford group $\mathcal{C}_3$, we multiply them repeatedly. The simplest result that one can think of being that a Clifford element is generated by a product of $n$ Hadamard gate and $\pi/4$ phase gate; here $n$ goes from 1 to $N$. What's $N$? We'd find out later.

For example, for $n=1$, we have two choices, either $H$ or $S$. That's two elements.
For $n=2$, we have $2\times 2$ choices. That's $HH, HS, SH, SS$. That's four more elements.
In general, for each $n$ we have $2^n$ choices. 

The follow lines of code gives us choices up to $n=12$. 

In [5]:
clifford_elements = []
for i in range(12):
    if i == 0:
        clifford_elements.append("H")
        clifford_elements.append("S")
    else: 
        for j in range(sum_index(i-1),sum_index(i)):
            clifford_elements.append(clifford_elements[j] + "H")
            clifford_elements.append(clifford_elements[j] + "S")

In [8]:
clifford_elements[-1]

'SSSSSSSSSSSS'

## Generate the group with matrix elements

Now we get the matrix form of these elements from their abstracted version.

In [9]:
clifford_element_matrix = []
for i in range(len(clifford_elements)):
    result = np.complex128(np.zeros([3,3]))
    for j in range(len(clifford_elements[i]),):
        if j == 0:
            if clifford_elements[i][j] == 'H': result += Hdm
            elif clifford_elements[i][j] == 'S': result += Sdg
        else:
            if clifford_elements[i][j] == 'H': result = result @ Hdm
            elif clifford_elements[i][j] =='S': result = result @ Sdg
    clifford_element_matrix.append(result)

In [10]:
clifford_element_matrix[-1]

array([[1.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.+0.00000000e+00j, 0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.-2.24912647e-15j]])

## Filtering out some equivalent matrices

Now the funny thing is that $\mathcal{C}_3$'s cardinality is 216. Here, how many elements do we have?

In [11]:
len(clifford_elements)

8190

It's 8190. Massive number innit? They are not entirely unique. In fact some of them are just some previous elements. This stems from the fact that the Clifford group is closed.

Let us call these elements "redundantor". The following lines of code will filter these redundantors out. 

In [13]:
clifford_matrix_filtered = []
clifford_matrix_filtered_index = []

for i in range(len(clifford_element_matrix)):
    if i == 0:
        clifford_matrix_filtered.append(clifford_element_matrix[i])
        clifford_matrix_filtered_index.append(i)
    else:
        flag = True
        for j in range(len(clifford_matrix_filtered)):
            if (np.round(clifford_element_matrix[i] - clifford_matrix_filtered[j], decimals = 10) == 0).all():
                flag = False
                break
            else: 
                continue
        if flag == True:
            clifford_matrix_filtered.append(clifford_element_matrix[i])
            clifford_matrix_filtered_index.append(i)
            
for i in range(len(clifford_matrix_filtered)):
    clifford_matrix_filtered[i] = np.round(clifford_matrix_filtered[i], decimals = 10)
    
Clifford3 = clifford_matrix_filtered
Clifford3_latex = [array_to_latex(mat) for mat in Clifford3]

In [14]:
len(Clifford3)

937

How many elements we're left with? 937. Wow. Massive reduction innit. We're getting closer to 216.

Now we realize that some elements in these 937 elements are just one previous element with an additional phase factor (or global phase if you will). We will eliminate them as well. The following lines of code do that.

In [15]:
def is_global_phase(mat_1, mat_2):
    '''
        Check whether matrix 1 and matrix are just one another but with an additional phase factor
    '''
    for i in range(3):
        if mat_1[0][i] != 0:
            arg_1 = np.angle(mat_1[0][i])
            arg_2 = np.angle(mat_2[0][i])
            break
        else:
            continue
    # identity = np.identity(3, np.complex128)
    # zeros = np.zeros([3,3], np.complex128)
    diff = mat_1 * np.exp(1j*arg_2) - mat_2 * np.exp(1j*arg_1)
    if (np.round(diff, decimals=10)==0).all():
        return True
    else:
        return False

In [37]:
matrix_1 = Hdm
matrix_2 = np.exp(1j * 10124*np.pi/3) * np.identity(3, np.complex128) @ Hdm

In [38]:
is_global_phase(matrix_1, matrix_2)

True

More specifically, for each $i$-th element we apply a brute force search over the entire set of 937 elements to see if any of them are equivalent to the $i$-th element up to a global phase.

In [41]:
global_phase = []
for i in range(len(Clifford3)):
    wrong = [i]
    for j in range(i+1, len(Clifford3)):
        if is_global_phase(Clifford3[i], Clifford3[j]):
            wrong.append(j)
    global_phase.append(wrong)

Now we group these equivalent elements into a "no_need" array.

In [45]:
no_need = set()
for i in range(len(global_phase)):
    for j in range(1, len(global_phase[i])):
        no_need.add(global_phase[i][j])
        
len(no_need)

721

There are 721 of them. Now we substract these "no_need" array from our unique set of Clifford elements

In [47]:
index = np.arange(0, len(Clifford3))

right_index = np.delete(index, list(no_need))
print(len(right_index))

Clifford3_unique = []
for i in right_index:
    Clifford3_unique.append(Clifford3[i])

216


Voila! 216 as wanted.

In [50]:
Clifford3_unique

[array([[ 0.57735027+0.j ,  0.57735027+0.j ,  0.57735027+0.j ],
        [ 0.57735027+0.j , -0.28867513+0.5j, -0.28867513-0.5j],
        [ 0.57735027+0.j , -0.28867513-0.5j, -0.28867513+0.5j]]),
 array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ],
        [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ],
        [ 0. +0.j       ,  0. +0.j       , -0.5+0.8660254j]]),
 array([[ 1.+0.j, -0.+0.j,  0.+0.j],
        [-0.+0.j,  0.+0.j,  1.-0.j],
        [ 0.+0.j,  1.-0.j,  0.+0.j]]),
 array([[ 0.57735027+0.j ,  0.57735027+0.j , -0.28867513+0.5j],
        [ 0.57735027+0.j , -0.28867513+0.5j,  0.57735027-0.j ],
        [ 0.57735027+0.j , -0.28867513-0.5j, -0.28867513-0.5j]]),
 array([[ 0.57735027+0.j ,  0.57735027+0.j ,  0.57735027+0.j ],
        [ 0.57735027+0.j , -0.28867513+0.5j, -0.28867513-0.5j],
        [-0.28867513+0.5j,  0.57735027-0.j , -0.28867513-0.5j]]),
 array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ],
        [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ]

Save them for use later!

In [53]:
import pickle

with open('./rb/data/Clifford3_unique.pkl', 'wb') as f:
    pickle.dump(Clifford3_unique, f)